In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from pymongo import MongoClient

C:\Users\rmsry\AppData\Roaming\Python\Python39\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [4]:
mongo_uri = "mongodb://hadoop-vm.internal.cloudapp.net:27017/ca2"

client = MongoClient(mongo_uri)
db = client["ca2"]
collection = db["vaccin_tweets_2_202004"]
collection_sentiment = db["vaccin_tweets_2_202004_sentiment"]

# Convert MongoDB collection to pandas DataFrame
data = pd.DataFrame(list(collection.find()))
data_sentiment = pd.DataFrame(list(collection_sentiment.find()))


In [ ]:
data.head()

In [3]:
data_sentiment.head()

""


In [ ]:
# Inner join
data = pd.merge(data, data_sentiment, on='_id', how='inner')

In [ ]:
# Add Date column and create index
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['date'] = data['timestamp'].dt.date

# Add Is Retweet column
data['is_retweet'] = data['text'].apply(lambda x: 1 if 'RT' in x else 0)

# Calculate avg_sentiment_score
df_grouped = data.groupby('date').agg(avg_sentiment_score=('s_probability', 'mean'))
df_grouped['retweet_rate'] = data.groupby('date')['is_retweet'].mean()

In [ ]:
df_grouped.head()

# Check stationationary

In [ ]:
# Check for stationarity with Dickey-Fuller test
result = adfuller(df_grouped['avg_sentiment_score'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])



> Data is stationary 

In [ ]:
startdate = pd.to_datetime("2017-7-7").date()
enddate = pd.to_datetime("2017-7-10").date()
df.loc[startdate:enddate]

In [ ]:
# Prepare train and test datasets
split_date = "2021-02-01"
train = df_grouped.where(df_grouped.date <= split_date)
test = df_grouped.where(df_grouped.date > split_date)

In [ ]:
# ARIMA
model = ARIMA(df_grouped['avg_sentiment_score'], order=(5,1,0)) # you will need to find the best parameters
model_fit = model.fit()
print(model_fit.summary())